In [1]:
import os

# define the root directory for the nfl code in $MLNLF_ROOT
codeDir = "".join([os.environ['MLNFL_ROOT'], os.path.sep])
dataRoot = "".join([codeDir, "data", os.path.sep])

os.chdir(codeDir)

print(codeDir)
print(dataRoot)

/Users/amit/repos/mlnfl/nfl/
/Users/amit/repos/mlnfl/nfl/data/


In [101]:
import madden
import numpy as np
import pandas

In [3]:
# location of lookup files
from referencedata import ReferenceData
lookupFiles = { 'teams' : {'file': 'nflTeams.csv' },
                'seasons' : {'file': 'seasons.csv' },
               }

lookupDir = "".join([dataRoot, 'lookup', os.path.sep])

print ("lookupFiles = %s" % lookupFiles)
print ("lookupDir = %s" % lookupDir)

lookupFiles = {'seasons': {'file': 'seasons.csv'}, 'teams': {'file': 'nflTeams.csv'}}
lookupDir = /Users/amit/repos/mlnfl/nfl/data/lookup/


In [4]:
# import reference data
reference_data = ReferenceData(lookupDir)
reference_data.teams_df.head()

,city,mascot,league,division,year
team,,,,,
Baltimore Ravens,NaN,Ravens,afc,north,NaN
New England Patriots,NaN,Patriots,afc,east,NaN
Tennessee Titans,NaN,Titans,afc,south,NaN
Atlanta Falcons,NaN,Falcons,nfc,south,NaN
Tampa Bay Buccaneers,NaN,Buccaneers,nfc,south,NaN


In [88]:
###  multi-season training
reload(madden)


# train on previous 3 yrs of data
testYear = [2013]
trainYears = range(testYear[0]-3,testYear[0]) 

# training data set - includes one extra year for prev yr record
seasons = np.array(trainYears)
print ("training seasons >> ", seasons)

('training seasons >> ', array([2010, 2011, 2012]))


In [89]:
path_to_lines = dataRoot + "lines/"
# 1 read all the games
df_all_historical_games = madden.readGamesAll(path_to_lines, seasons)
# 2 - compile season record for all teams
df_all_teams = madden.seasonRecord(df_all_historical_games, reference_data)
#3 - apply season records and compute other fields for all games
df_all_historical_games = madden.processGames(df_all_historical_games, df_all_teams, reference_data)
# 4 - remove extra year of data
df_all_historical_games = df_all_historical_games[df_all_historical_games.season.isin(seasons)]


In [90]:
# use different test set
season_test = np.array(testYear) # should be only one year
# 1 - read all the games
dfGamesTest = madden.readGamesAll(path_to_lines, season_test)
# 2 - compile season record for all teams
dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data)
# 3 - apply season records and compute other fields for all games
dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data)
# 4 - remove extra year of data
dfGamesTest = dfGamesTest[dfGamesTest.season.isin(season_test)]


In [91]:
# run the classifier
from sklearn import linear_model
lr_classifier = linear_model.LogisticRegression(C=1e5)
lr_trained_classifier = madden.runScikitClassifier(df_all_historical_games, madden.FEATURE_COLUMNS, lr_classifier)



In [92]:
week_number = 1
dfTest = dfGamesTest[dfGamesTest.gameWeek == week_number]
dfTest

,Date,Visitor,Visitor Score,Home Team,Home Score,Line,Total Line,season,finalLine,yahooLine,...,homeWin,favoredWin,gameWeek,homeRecord,visitorRecord,favoredRecord,underdogRecord,prevFavoredRecord,prevUnderdogRecord,absLine
8174,9/5/13,Baltimore Ravens,27.0,Denver Broncos,49.0,7.5,49.5,2013,NaN,NaN,...,1,1,1,0.0,0.0,0.0,0.0,0.8125,0.6250,7.5
8175,9/8/13,New England Patriots,23.0,Buffalo Bills,21.0,-10.5,51.5,2013,NaN,NaN,...,0,1,1,0.0,0.0,0.0,0.0,0.7500,0.3750,10.5
8176,9/8/13,Tennessee Titans,16.0,Pittsburgh Steelers,9.0,6.0,42,2013,NaN,NaN,...,0,0,1,0.0,0.0,0.0,0.0,0.5000,0.3750,6.0
8177,9/8/13,Atlanta Falcons,17.0,New Orleans Saints,23.0,3.5,56,2013,NaN,NaN,...,1,1,1,0.0,0.0,0.0,0.0,0.4375,0.8125,3.5
8178,9/8/13,Tampa Bay Buccaneers,17.0,New York Jets,18.0,-6.0,39,2013,NaN,NaN,...,1,0,1,0.0,0.0,0.0,0.0,0.4375,0.3750,6.0
8179,9/8/13,Kansas City Chiefs,28.0,Jacksonville Jaguars,2.0,-4.5,43,2013,NaN,NaN,...,0,1,1,0.0,0.0,0.0,0.0,0.1250,0.1250,4.5
8180,9/8/13,Cincinnati Bengals,21.0,Chicago Bears,24.0,3.0,42,2013,NaN,NaN,...,1,1,1,0.0,0.0,0.0,0.0,0.6250,0.6250,3.0
8181,9/8/13,Miami Dolphins,23.0,Cleveland Browns,10.0,2.5,40.5,2013,NaN,NaN,...,0,0,1,0.0,0.0,0.0,0.0,0.3125,0.4375,2.5
8182,9/8/13,Seattle Seahawks,12.0,Carolina Panthers,7.0,-3.5,45,2013,NaN,NaN,...,0,1,1,0.0,0.0,0.0,0.0,0.6875,0.4375,3.5
8183,9/8/13,Minnesota Vikings,24.0,Detroit Lions,34.0,4.5,47.5,2013,NaN,NaN,...,1,1,1,0.0,0.0,0.0,0.0,0.2500,0.6250,4.5


In [93]:
# apply results of logistic regression to the test set
df_lr_predict = madden.predictGames(dfTest, lr_trained_classifier, madden.FEATURE_COLUMNS)


In [94]:
df_lr_predict

,Date,Visitor,Visitor Score,Home Team,Home Score,Line,Total Line,season,finalLine,yahooLine,...,homeRecord,visitorRecord,favoredRecord,underdogRecord,prevFavoredRecord,prevUnderdogRecord,absLine,predict_proba,predict_proba_abs,predictWin
8174,9/5/13,Baltimore Ravens,27.0,Denver Broncos,49.0,7.5,49.5,2013,NaN,NaN,...,0.0,0.0,0.0,0.0,0.8125,0.6250,7.5,0.751582,0.251582,1
8175,9/8/13,New England Patriots,23.0,Buffalo Bills,21.0,-10.5,51.5,2013,NaN,NaN,...,0.0,0.0,0.0,0.0,0.7500,0.3750,10.5,0.780155,0.280155,1
8176,9/8/13,Tennessee Titans,16.0,Pittsburgh Steelers,9.0,6.0,42,2013,NaN,NaN,...,0.0,0.0,0.0,0.0,0.5000,0.3750,6.0,0.670814,0.170814,0
8177,9/8/13,Atlanta Falcons,17.0,New Orleans Saints,23.0,3.5,56,2013,NaN,NaN,...,0.0,0.0,0.0,0.0,0.4375,0.8125,3.5,0.590758,0.090758,1
8178,9/8/13,Tampa Bay Buccaneers,17.0,New York Jets,18.0,-6.0,39,2013,NaN,NaN,...,0.0,0.0,0.0,0.0,0.4375,0.3750,6.0,0.635900,0.135900,0
8179,9/8/13,Kansas City Chiefs,28.0,Jacksonville Jaguars,2.0,-4.5,43,2013,NaN,NaN,...,0.0,0.0,0.0,0.0,0.1250,0.1250,4.5,0.540515,0.040515,1
8180,9/8/13,Cincinnati Bengals,21.0,Chicago Bears,24.0,3.0,42,2013,NaN,NaN,...,0.0,0.0,0.0,0.0,0.6250,0.6250,3.0,0.572158,0.072158,1
8181,9/8/13,Miami Dolphins,23.0,Cleveland Browns,10.0,2.5,40.5,2013,NaN,NaN,...,0.0,0.0,0.0,0.0,0.3125,0.4375,2.5,0.526931,0.026931,0
8182,9/8/13,Seattle Seahawks,12.0,Carolina Panthers,7.0,-3.5,45,2013,NaN,NaN,...,0.0,0.0,0.0,0.0,0.6875,0.4375,3.5,0.533531,0.033531,1
8183,9/8/13,Minnesota Vikings,24.0,Detroit Lions,34.0,4.5,47.5,2013,NaN,NaN,...,0.0,0.0,0.0,0.0,0.2500,0.6250,4.5,0.612135,0.112135,1


In [95]:


df_lr_predict.groupby('gameWeek').agg({'predict_proba':max})

,predict_proba
gameWeek,
1,0.838624


In [96]:
weekly_picks

,Date,Visitor,Visitor Score,Home Team,Home Score,Line,Total Line,season,finalLine,yahooLine,...,predict_proba,predict_proba_abs,predictWin,lineGuess,probaGuess,probaAbsGuess,lineScore,probaScore1,probaScore2,probaScore3
8494,10/5/14,St Louis Rams,28.0,Philadelphia Eagles,34.0,7.0,NaN,2014,NaN,NaN,...,0.817255,0.317255,1,14.0,16.0,16.0,14.0,16.0,16.0,16.0
8503,10/5/14,New York Jets,0.0,San Diego Chargers,31.0,6.5,NaN,2014,NaN,NaN,...,0.810941,0.310941,1,10.0,15.0,15.0,10.0,15.0,15.0,15.0
8496,10/5/14,Tampa Bay Buccaneers,31.0,New Orleans Saints,37.0,10.0,NaN,2014,NaN,NaN,...,0.797850,0.297850,1,16.0,14.0,14.0,16.0,14.0,14.0,14.0
8491,10/2/14,Minnesota Vikings,10.0,Green Bay Packers,42.0,9.0,NaN,2014,NaN,NaN,...,0.783105,0.283105,1,15.0,13.0,13.0,15.0,13.0,13.0,13.0
8498,10/5/14,Buffalo Bills,17.0,Detroit Lions,14.0,7.0,NaN,2014,NaN,NaN,...,0.762872,0.262872,0,13.0,12.0,12.0,0.0,0.0,0.0,0.0
8501,10/5/14,Arizona Cardinals,20.0,Denver Broncos,41.0,7.0,NaN,2014,NaN,NaN,...,0.751104,0.251104,1,12.0,11.0,11.0,12.0,11.0,11.0,11.0
8505,10/6/14,Seattle Seahawks,27.0,Washington Redskins,17.0,-7.0,NaN,2014,NaN,NaN,...,0.738102,0.238102,1,11.0,10.0,10.0,11.0,10.0,10.0,10.0
8502,10/5/14,Kansas City Chiefs,17.0,San Francisco 49ers,22.0,6.0,NaN,2014,NaN,NaN,...,0.717164,0.217164,1,9.0,9.0,9.0,9.0,9.0,9.0,9.0
8500,10/5/14,Pittsburgh Steelers,17.0,Jacksonville Jaguars,9.0,-6.0,NaN,2014,NaN,NaN,...,0.700786,0.200786,1,8.0,8.0,8.0,8.0,8.0,8.0,8.0
8497,10/5/14,Houston Texans,17.0,Dallas Cowboys,20.0,5.5,NaN,2014,NaN,NaN,...,0.609331,0.109331,1,7.0,7.0,7.0,7.0,7.0,7.0,7.0


In [97]:
reload(madden)
df_all_picks = madden.rankGames(df_lr_predict, reference_data, season_test[0])


In [113]:
def get_games(yr):
    # train on previous 3 yrs of data
    testYear = [yr]
    trainYears = range(testYear[0]-3,testYear[0]) 

    # training data set - includes one extra year for prev yr record
    seasons = np.array(trainYears)

    path_to_lines = dataRoot + "lines/"
    # 1 read all the games
    df_all_historical_games = madden.readGamesAll(path_to_lines, seasons)
    # 2 - compile season record for all teams
    df_all_teams = madden.seasonRecord(df_all_historical_games, reference_data)
    #3 - apply season records and compute other fields for all games
    df_all_historical_games = madden.processGames(df_all_historical_games, df_all_teams, reference_data)
    # 4 - remove extra year of data
    df_all_historical_games = df_all_historical_games[df_all_historical_games.season.isin(seasons)]

    # use different test set
    season_test = np.array(testYear) # should be only one year
    # 1 - read all the games
    dfGamesTest = madden.readGamesAll(path_to_lines, season_test)
    # 2 - compile season record for all teams
    dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data)
    # 3 - apply season records and compute other fields for all games
    dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data)
    # 4 - remove extra year of data
    dfGamesTest = dfGamesTest[dfGamesTest.season.isin(season_test)]
    
    return df_all_historical_games, dfGamesTest



In [116]:
df_years = pandas.DataFrame(columns=['year', 'first_loss', 'second_loss'])

for yr in range(2010,2016):
    
    df_all_historical_games, dfGamesTest = get_games(yr)
    
    survivor_picks = set()
    loss_week = list()

    # run the classifier
    lr_classifier = linear_model.LogisticRegression(C=1e5)
    lr_trained_classifier = madden.runScikitClassifier(df_all_historical_games, madden.FEATURE_COLUMNS, lr_classifier)

    for w in range(1,18):
        dfTest = dfGamesTest[dfGamesTest.gameWeek == w]
        df_lr_predict = madden.predictGames(dfTest, lr_trained_classifier, madden.FEATURE_COLUMNS)
        df_all_picks = madden.rankGames(df_lr_predict, reference_data, yr)
        weekly_picks = df_all_picks.sort('predict_proba', ascending=False)
        ipick = 0
        losses = 0
        while weekly_picks.irow(ipick).favorite in survivor_picks:
            ipick += 1

        pick = weekly_picks.irow(ipick).favorite
        survivor_picks.add(pick)
        pick_rank = weekly_picks.irow(ipick).probaGuess
        pick_result = weekly_picks.irow(ipick).favoredWin
        if pick_result == 0:
            loss_week.append(int(w))

        print "week %d: pick: %s, rank=%d, won=%d" % (w, pick, pick_rank, pick_result)

    if len(loss_week)< 2:
        loss_week.append(0)

    df_years = df_years.append( {'year':int(yr), 'first_loss': loss_week[0], 'second_loss':loss_week[1]} , ignore_index=True)
df_years

/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/__main__.py:18: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/__main__.py:21: FutureWarning: irow(i) is deprecated. Please use .iloc[i]
/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/__main__.py:24: FutureWarning: irow(i) is deprecated. Please use .iloc[i]
/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/__main__.py:26: FutureWarning: irow(i) is deprecated. Please use .iloc[i]
/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/__main__.py:27: FutureWarning: irow(i) is deprecated. Please use .iloc[i]


week 1: pick: Tennessee Titans, rank=16, won=1
week 2: pick: Green Bay Packers, rank=16, won=1
week 3: pick: New England Patriots, rank=16, won=1
week 4: pick: New Orleans Saints, rank=15, won=1
week 5: pick: New York Jets, rank=16, won=1
week 6: pick: Pittsburgh Steelers, rank=16, won=1
week 7: pick: Baltimore Ravens, rank=16, won=1
week 8: pick: Kansas City Chiefs, rank=16, won=1
week 9: pick: Chicago Bears, rank=13, won=1
week 10: pick: New York Giants, rank=16, won=0
week 11: pick: San Diego Chargers, rank=13, won=1
week 12: pick: Cleveland Browns, rank=13, won=1
week 13: pick: Philadelphia Eagles, rank=13, won=1
week 14: pick: Atlanta Falcons, rank=15, won=1
week 15: pick: Oakland Raiders, rank=14, won=1
week 16: pick: Tampa Bay Buccaneers, rank=13, won=1
week 17: pick: Indianapolis Colts, rank=13, won=1
week 1: pick: San Diego Chargers, rank=16, won=1
week 2: pick: Detroit Lions, rank=16, won=1
week 3: pick: Pittsburgh Steelers, rank=15, won=1
week 4: pick: Green Bay Packers, ran

,year,first_loss,second_loss
0,2010.0,10.0,0.0
1,2011.0,5.0,10.0
2,2012.0,3.0,0.0
3,2013.0,2.0,10.0
4,2014.0,6.0,0.0
5,2015.0,4.0,10.0


In [99]:
w = 2
dfTest = dfGamesTest[dfGamesTest.gameWeek == w]
df_lr_predict = madden.predictGames(dfTest, lr_trained_classifier, madden.FEATURE_COLUMNS)
df_all_picks = madden.rankGames(df_lr_predict, reference_data, season_test[0])
weekly_picks = df_all_picks.sort('predict_proba', ascending=False)
weekly_picks

/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Date,Visitor,Visitor Score,Home Team,Home Score,Line,Total Line,season,finalLine,yahooLine,...,predict_proba,predict_proba_abs,predictWin,lineGuess,probaGuess,probaAbsGuess,lineScore,probaScore1,probaScore2,probaScore3
8191,9/15/13,San Diego Chargers,33.0,Philadelphia Eagles,30.0,7.5,54,2013,NaN,NaN,...,0.879920,0.379920,0,13.0,16.0,16.0,0.0,0.0,0.0,0.0
8199,9/15/13,Minnesota Vikings,30.0,Chicago Bears,31.0,6.5,42,2013,NaN,NaN,...,0.857530,0.357530,1,11.0,15.0,15.0,11.0,15.0,15.0,15.0
8190,9/12/13,New York Jets,10.0,New England Patriots,13.0,11.5,43.5,2013,NaN,NaN,...,0.814829,0.314829,1,16.0,14.0,14.0,16.0,14.0,14.0,14.0
8203,9/15/13,Denver Broncos,41.0,New York Giants,23.0,-4.0,55.5,2013,NaN,NaN,...,0.779521,0.279521,1,6.0,13.0,13.0,6.0,13.0,13.0,13.0
8197,9/15/13,Washington Redskins,20.0,Green Bay Packers,38.0,8.0,50,2013,NaN,NaN,...,0.769824,0.269824,1,14.0,12.0,12.0,14.0,12.0,12.0,12.0
8200,9/15/13,New Orleans Saints,16.0,Tampa Bay Buccaneers,14.0,-4.0,48.5,2013,NaN,NaN,...,0.749235,0.249235,1,7.0,11.0,11.0,7.0,11.0,11.0,11.0
8193,9/15/13,Tennessee Titans,24.0,Houston Texans,30.0,9.0,43,2013,NaN,NaN,...,0.736522,0.236522,1,15.0,10.0,10.0,15.0,10.0,10.0,10.0
8201,9/15/13,Detroit Lions,21.0,Arizona Cardinals,25.0,-2.0,48.5,2013,NaN,NaN,...,0.685111,0.185111,0,1.0,9.0,9.0,0.0,0.0,0.0,0.0
8205,9/16/13,Pittsburgh Steelers,10.0,Cincinnati Bengals,20.0,6.5,41,2013,NaN,NaN,...,0.682700,0.182700,1,10.0,8.0,8.0,10.0,8.0,8.0,8.0
8192,9/15/13,Cleveland Browns,6.0,Baltimore Ravens,14.0,7.0,44,2013,NaN,NaN,...,0.682296,0.182296,1,12.0,7.0,7.0,12.0,7.0,7.0,7.0


In [119]:
df_years

,year,first_loss,second_loss
0,2010.0,10.0,0.0
1,2011.0,5.0,10.0
2,2012.0,3.0,0.0
3,2013.0,2.0,10.0
4,2014.0,6.0,0.0
5,2015.0,4.0,10.0
